<img src="https://static.wixstatic.com/media/a9ca5e_825bd4d39e7d468faf735b801fa3dea4~mv2.png/v1/fill/w_1458,h_246,al_c,usm_0.66_1.00_0.01/a9ca5e_825bd4d39e7d468faf735b801fa3dea4~mv2.png" width="200">


 # __Notebook voor het importeren van 3.* auditfiles__

 
 __Auteur:__ Melvin Folkers (Yellow Stacks B.V.)<br>
 __Versie:__ 06 augustus 2018<br>
 __Doel:__ Het importeren van auditfiles 3.* <br>
 ***

__overzicht van paragrafen in deze notebook__
 > 1. Parameters instellen<br>  
 > 2. functies<br>
 > 3. Auditfile tabellen importeren<br>
 > 4. Auditfile preppen<br>
 > 5. Auditfile exporteren<br>
 

__introductie__

Het doel van deze notebook is om auditfiles van het type 3 om te zetten in een dataset van grootboekmutaties.<br> 
De grootboekmutaties worden voorzien van informatie van de relaties, btwcodes, grootboek,dagboek, -en bedrijfsinformatie. <br>
In het script worden er ook validatiechecks gedaan op de ingelezen regels. <br>
De mutaties worden vervolgens geexporteerd naar een excelbestand.

Het script is gemaakt om __1 auditfile in te lezen__, ter demonstratie van wat python kan betekenen voor collega's werkzaam in de financiele branche (te denken aan: audit / tax / controlling).<br>
Dit soort scripts kunnen doorontwikkeld worden opdat meer auditfiles tergelijk kunnen worden ingelezen.<br>
Zorg ervoor dat wanneer je het script wil draaien, dat je de auditfile in dezelfde map zet als deze notebook.

De installatie van jupyter notebook staat goed gedocumenteerd op de website van jupyter: http://jupyter.org/install




# 1. Parameters instellen

als je voor de eerste keer jupyter notebook gebruikt moeten er nog wat bibliotheken (packages) geinstalleerd worden. Door onderstaande code te runnen met de Install parameter op <font color="green">__True__</font>.<br>
Wanneer je dat hebt gedaan kan je de waarde weer op <font color="green">__False__</font> zetten, omdat je dit maar 1 keer hoeft te doen.

### 1.0 installeren van de benodigde bibliotheken

In [3]:
install = False

if install == False : None
else : 
    !pip install numpy
    !pip install matplotlib
    !pip install pandas
    !pip install xml.etree.ElementTree
    !pip install matplotlib

### 1.1 Notebook settings
__instellingen voor gebruik van deze jupyter notebook__<br>
deze instellingen zorgen ervoor dat de notebook het hele scherm vult en dat er (meerdere) grafieken getoond kunnen worden.

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # maakt de jupyter notebook cellen 100% breed

%matplotlib inline 
# voor het plotten van grafieken

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # om meerdere grafieken/tabellen in 1 cel te kunnen laten zien.

### 1.2 Importeren van bibliotheken
De bibliotheken die nodig zijn voor het script zijn in de eerste stap geinstalleerd. Als je dit nog niet gedaan hebt zal er hier een error komen die aangeeft dat de bibliotheek ontbreekt.

In [5]:
# importeren van de bibliotheken
import sys, os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

from datetime import datetime

***
# 2. Functies
__functies voor importeren van diverse lagen in de auditfile__ <br><br>
De functies die hieronder zijn geschreven zijn gemaakt om de meerder lagen van de auditfile door te lezen.<br>
Functies moeten altijd bovenaan het script worden gezet, omdat je deze later gaat aanroepen.<br>
Het echte script begint dus pas in __hoofdstuk 3__. let op dat je het onderstaande script runt tot hoofdstuk 3.

### 2.1 Functies voor het parsen van data

In [6]:
def parse_info(root): # Bedoeld om alle informatie uit de root van de auditfile te halen. Er moet ook een manier zijn om in dit 1 functie te kunnen schrijven, maar voor nu werkt dit ook.
   
    recordcontent = dict()
    
    for child in root:
        columnname = child.tag.replace(ns,'')
        columnvalue = child.text
        
        if len(child) == 0:
            recordcontent[columnname] = columnvalue     
            
        else: continue
            
    return recordcontent  

In [7]:
def stamtabellen(root) :    # bedoeld om subtabellen als customer supplier uit te lezen.
    
    currentrow = 0
    records = dict()

    for child in root:

        module = child.tag.replace(ns,'')
        recordcontent = dict()

        for subchild in child:

            if len(subchild) != 0:
                for subsubchild in subchild:
                    columnvalue = subsubchild.text
                    columnname = subsubchild.tag.replace(ns,'')
                    recordcontent[columnname] = columnvalue

                    if len(subsubchild) != 0: print('let op! nog een sublaag gevonden')
            else:
                columnvalue =subchild.text
                columnname = subchild.tag.replace(ns,'')
                recordcontent[columnname] = columnvalue

        records[currentrow] = recordcontent
        currentrow +=1

    df = pd.DataFrame(records).transpose()
    return df

In [8]:
def tags_in_module(modules): # functie om de tags van de xmllaag uit te lezen. Dit zijn soms namen van submodules maar kunnen ook kolomnamen zijn.
    tag = dict()
    
    for submodule in modules:
        tagname = submodule.tag.replace(ns,'')
        tag[tagname] = tag.get(tagname, 0) + 1
    return tag

### 2.2 Functies voor het converteren van data

In [9]:
def accounttype(dataframe): # functie voor het bepalen van categorie Balans of Winst & Verlies

    conditions = [
        (dataframe['accTp'] == 'P'),
        (dataframe['accTp'] == 'B')]

    choices = ['Winst & verlies','Balans']

    dataframe['accounttype'] = np.select(conditions, choices, default= 'onbekende balanstype' + dataframe['accTp'] )


In [10]:
def journaltype(dataframe): # functie voor het bepalen van de dagboektypes.

    conditions = [
        (dataframe['jrnTp'] == 'Z'),
        (dataframe['jrnTp'] == 'B'),
        (dataframe['jrnTp'] == 'P'),
        (dataframe['jrnTp'] == 'O'),
        (dataframe['jrnTp'] == 'C'),
        (dataframe['jrnTp'] == 'M'),
        (dataframe['jrnTp'] == 'Y'),
        (dataframe['jrnTp'] == 'S')]

    choices = ['memoriaal', 'bankboek' , 'inkoopboek' , 'open/sluit balans', 'kasboek', 'memoriaal', 'salaris', 'verkoopboek']

    dataframe['journaltype'] = np.select(conditions, choices, default= 'onbekend dagboek' )

In [11]:
def vat_amount(dataframe): # functie die de waarde van kolom vat_amount in het goede formaat staat. de kolom wordt hernoemt naar vat_amount

    vat_amount_raw = dataframe['vatAmnt'].astype(float)
    
    conditions = [
        (dataframe['vatAmntTp'] == 'C'),
        (dataframe['vatAmntTp'] == 'D')]

    choices = [-1,1]

    dataframe['vat_amount'] = np.select(conditions, choices, default= 1 ) * vat_amount_raw


In [12]:
def amount(dataframe): # functie die de waarde van kolom amount in het goede formaat staat. de kolom wordt hernoemt naar amount



    amount_raw = dataframe['amnt'].astype(float)
    
    conditions = [
        (dataframe['amntTp'] == 'C'),
        (dataframe['amntTp'] == 'D')]

    choices = [-1,1]

    dataframe['amount'] = np.select(conditions, choices, default= 1 ) * amount_raw


***
# 3. Importeren van de auditfile
_starten met importeren van de auditfile data_

### 3.1 lees de auditfile in met de xml parser

__selecteren van (demo) auditfile__<br>

Let op: <br><br>

in dit geval is een voorbeeld auditfile gebruikt ten behoeve van een demo. <br>
Mocht je het op een andere auditfile willen proberen dan kan je simpelweg voor de waarde 'file' de naam van de auditfile neerzetten.<br>
Het is wel belangrijk dat de auditfile in dit geval op de locatie van dit de demo auditfiles staat. <br>
Mocht het op een andere locatie staan dan kan je ook het relatieve pad invullen (bv. 'C:/file.xaf')<br>

In [13]:
file  = 'ExactOnline_V32_2016.xaf'

In [17]:


tree = ET.parse(file) # mocht je een andere map hebben dan kan je het pad hier aanpassen
root = tree.getroot()
namespaces = {'xsd':"http://www.w3.org/2001/XMLSchema", 'xsi':"http://www.w3.org/2001/XMLSchema-instance" }


_stel namespace vast_

In [18]:
ns_raw =  root.tag.split('{')[1].split('}')[0]
ns = '{'+ ns_raw + '}'
namespaces['af'] = ns_raw
namespaces

{'xsd': 'http://www.w3.org/2001/XMLSchema',
 'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 'af': 'http://www.auditfiles.nl/XAF/3.2'}

### 3.2 Maak onderscheid tussen header en company subledgers

_splitst de auditfile op in 3 blokken._
> - header
> - company
> - transactions

In [19]:
header    = root.find('af:header',namespaces) # zoekt in de xml naar de tag header
company   = root.find('af:company', namespaces) # zoekt in de xml naar de tag company
transactions = root.find('af:company/af:transactions', namespaces) # zoekt in de xml naar de tag company/transactions (rekening houdend met de prefix van de namespaces)

_leest de eerste laag van het xml niveau in. <br>Dit kunnen we zien als de metadata van de auditfile_

In [20]:
# zoek per laag naar de informatie die uniek is voor de auditfile.
headerinfo = pd.DataFrame(parse_info(header), index = [0])
companyinfo = pd.DataFrame(parse_info(company), index = [0])
transactioninfo = pd.DataFrame(parse_info(transactions), index = [0])

# plak al deze informatie over de metadata van de auditfiles in 1 dataframe. Dit past in 1 regel.

af_info = pd.concat([headerinfo, companyinfo, transactioninfo], axis = 1)

af_info['file'] = file
af_info 

,curCode,dateCreated,endDate,fiscalYear,softwareDesc,startDate,companyIdent,companyName,taxRegIdent,taxRegistrationCountry,linesCount,totalCredit,totalDebit,file
0,EUR,2017-04-20,2016-12-31,2016,Exact Online,2016-01-01,1,Toro Rosso BV,None,NL,167,999954.06,999954.06,ExactOnline_V32_2016.xaf


### 3.3 Parsen van data van de stamtabellen

_aanmaken van de volgende subtabellen:_
> - periods <br>
> - customer suppliers <br>
> - vatcodes <br>
> - generalLedger <br>
> - basics<br>

In [21]:
periods = stamtabellen(company.findall('af:periods/af:period',namespaces))
custsup = stamtabellen(company.findall('af:customersSuppliers/af:customerSupplier',namespaces))
vatcode = stamtabellen(company.findall('af:vatCodes/af:vatCode',namespaces))
genledg  = stamtabellen(company.findall('af:generalLedger/af:ledgerAccount',namespaces))
basics  = stamtabellen(company.findall('af:generalLedger/af:basics',namespaces))

In [22]:
# ontdubbel vat ID's die een claim en pay account hebben. deze kunnen later voor een verdubbeling van de data leiden.
# wel nemen we alle informatie mee door de twee tabellen te splitsen en vervolgens op vatID aan elkaar te joinen.

claim = vatcode[(['vatID', 'vatDesc','vatToClaimAccID'])]
claim = claim[pd.isnull(claim['vatToClaimAccID']) == False]

pay = vatcode[(['vatID', 'vatDesc','vatToPayAccID'])]
pay = pay[pd.isnull(pay['vatToPayAccID']) == False]

vatcode = pd.merge(claim,pay, on = ['vatID', 'vatDesc'], how ='outer')

In [23]:
basics.head()
genledg.head()
vatcode.head()
periods.head()
custsup.head()

,basicDesc,basicID,basicType
0,joris.joppe@gmail.com,1849800,30


,accDesc,accID,accTp,leadCode,leadDescription,leadReference
0,Geplaatst kapitaal,0310,B,11,Materiële vaste activa,BEivGokGea
1,473677091,1101,B,16,Liquide middelen,BLimBanRba
2,Afschrijvingskosten gebouwen,5110,P,42,Afschrijvingen,WAfsAmvBeg
3,Resultaat verkoop activa,5300,P,42,Afschrijvingen,WAfsRvmBei
4,Overige bedrijfsopbrengsten,8900,P,30,Omzet,WOmzNopOlh


,vatID,vatDesc,vatToClaimAccID,vatToPayAccID
0,1,"BTW laag tarief, exclusief",1520,1510
1,10,Verwerving binnen de EU,1520,1512
2,11,BTW prive gebruik,1520,1513
3,12,BTW overige tarieven,1520,1500
4,2,"BTW hoog tarief, exclusief",1520,1500


,endDatePeriod,periodNumber,startDatePeriod
0,2016-01-31,1,2016-01-01
1,2016-02-29,2,2016-02-01
2,2016-03-31,3,2016-03-01
3,2016-04-30,4,2016-04-01
4,2016-05-31,5,2016-05-01


,bankAccNr,bankIdCd,city,commerceNr,contact,country,custSupID,custSupName,custSupTp,eMail,fax,postalCode,streetname,taxRegIdent,taxRegistrationCountry,telephone,website
0,NaN,NaN,NaN,NaN,NaN,NL,1,ABC Producten & Diensten B.V.,B,NaN,NaN,NaN,NaN,NaN,NL,NaN,NaN
1,2445588,PSTBNL21,Apeldoorn,NaN,NaN,NL,10,Belastingdienst,S,NaN,NaN,NaN,NaN,NaN,NL,NaN,www.belastingdienst.nl
2,NaN,NaN,'S-GRAVENHAGE,271376310000,NaN,NL,11,Mijn Magazijn B.V.,S,NaN,NaN,2583AK,Westduinweg 262,NaN,NL,070 3559269,NaN
3,NaN,NaN,ROTTERDAM,244773190000,NaN,NL,12,Warmtebedrijf Eneco Delft B.V.,S,NaN,NaN,3068AZ,G.H. Betzweg 1,NaN,NL,088 8953736,www.warmteindelft.nl
4,NaN,NaN,UTRECHT,370267060000,NaN,NL,13,Ziggo B.V.,S,NaN,NaN,3542AB,Atoomweg 100,NaN,NL,088 7170000,www.ziggo.nl


### 3.4 Parsen van data van de journals

In [24]:


journals = company.findall('af:transactions/af:journal', namespaces)
journal_df = pd.DataFrame()

for journal in journals: # importeert de aanwezig dagboeken. Per dagboek zijn de transacties in een sublaag te vinden.
    jrninfo = dict()
    
    for records in journal:
        if len(records) == 0:
            columnnames = records.tag.replace(ns,'')
            columnvalues = records.text
            jrninfo[columnnames] = columnvalues
    journal_df = journal_df.append(jrninfo, ignore_index = True)


journaltype(journal_df) # format het juiste dagboektype
journal_df = journal_df.drop(['jrnTp'] , axis = 1)

journal_df

,desc,jrnID,offsetAccID,bankAccNr,journaltype
0,Kasboek,10,1000,NaN,kasboek
1,Bankboek,20,1100,1111111111,bankboek
2,473677091,21,1101,473677091,bankboek
3,bank test,22,1102,0444144803,bankboek
4,Inkoopboek,60,1600,NaN,inkoopboek
5,Verkoopboek,70,1300,NaN,verkoopboek
6,Memoriaal,90,NaN,NaN,memoriaal
7,Activamutaties,95,NaN,NaN,memoriaal


### 3.5 Parsen van de transactions in de journals
In de journals zitten meerdere transactions. we willen van alle journals alle transactions hebben:
> journal 1 <br>
> journal 2 <br>
> journal 3 <br>
>> transaction 1 <br>
>> transaction 2 <br>
>> transaction 3 <br>
>>> transactionline 1  <font color='green'>kolomnamen, kolomnwaardes</font> <br>
>>> transactionline 2  <font color='green'>kolomnamen, kolomnwaardes</font> <br>
>>> transactionline 3  <font color='green'>kolomnamen, kolomnwaardes</font> <br>


In [25]:
transactions_df = pd.DataFrame()

total_records = list()
record_dict = dict()

for journal in journals: # voor alle dagboeken in de auditfile
    
    
    for records in journal: # voor de alle records die in het dagboek zitten
        
        if len(records) == 0:
            columnnames = records.tag.replace(ns,'')
            columnvalues = records.text
            record_dict[columnnames] = columnvalues

        else:
            for record in records: # voor alle velden in de record
                if len(record) == 0:
                    columnnames = record.tag.replace(ns,'')
                    columnvalues = record.text
                    record_dict[columnnames] = columnvalues # de kolomnaam en kolomwaarde van dit record

                else:

                    for subfields in record: # soms zit de informatie nog een laag dieper.
                        if len(subfields) == 0:
                            columnnames = subfields.tag.replace(ns,'')
                            columnvalues = subfields.text
                            record_dict[columnnames] = columnvalues #  de kolomnaam en kolomwaarde van dit record

                        else: 

                            for subfields_1 in subfields: # check of er nog een laag dieper is. als dit zo is krijg je terug dat er nog een sublaag is gevonden. Normaliter zal deze if statement nooit getriggered worden.
                                if len(subfields_1) == 0:
                                    columnnames = subfields_1.tag.replace(ns,'')
                                    columnvalues = subfields_1.text
                                    record_dict[columnnames] = columnvalues
                                else : print('nog een sublaag!')

                    
                    total_records.append(record_dict.copy()) # plak de record aan de totaal tabel.

transactions_df = transactions_df.append(total_records, ignore_index = True)


***
# 4. Auditfile preppen

### 4.1 accounttype toevoegen aan general ledger

In [26]:
accounttype(genledg) # bepaal de types Winst & Verlies of Balans --> zie functies aan het begin van het script.

### 4.2 Transacties het juiste dataformat geven
_aanmaken van nieuwe velden en aanpassen van datatypes_

In [31]:
tr = transactions_df 

amount(tr) # zet het amountveld in het juiste formaat --> zie functies

tr = tr.drop(['amnt', 'amntTp', ], axis=1)


tr['effDate'] = pd.to_datetime(tr['effDate'])
tr['trDt'] = pd.to_datetime(tr['trDt'])

if 'vatAmnt' in tr.columns:
    vat_amount(tr)  # zet het vat_amount veld in het juiste formaat --> zie functies
    tr = tr.drop(['vatAmnt', 'vatAmntTp'], axis=1)

else:
    print('geen vat amount!')
    tr['vatID'] = None


### 4.3 Tabellen aan elkaar koppelen
_De volgende stamtabellen worden aan elkaar gelinkt op unieke ID_
   1. periods
   2. vatcodes
   3. customers suppliers
   4. general ledger
   5. journalinfo
   6. auditfile info
   

In [32]:
    # join met periods
    temp_1 = pd.merge(tr, periods, on ='periodNumber', how='left')

    # join met vatcodes

    if len(vatcode) != 0 :
        temp_2 = pd.merge(temp_1, vatcode, on='vatID', how='left')
    else:
        temp_2 = temp_1

    # join met customersuppliers
    if 'custSupID' in temp_2.columns and len(custsup) != 0:
        temp_3 = pd.merge(temp_2, custsup.add_prefix('cs_'), left_on='custSupID' , right_on = 'cs_custSupID', how='left')
    else:
        temp_3 = temp_2

    # join met generalLedger
    temp_4 = pd.merge(temp_3, genledg, on='accID' , how='left')

    # join met journalinfo
    temp_5= pd.merge(temp_4, journal_df.add_prefix('jrn_'),left_on = 'jrnID', right_on='jrn_jrnID' , how='left')

    
    # Plak de filename aan de transacties
    temp_5['file'] = file
    
    # sommige zullen de volgende stap van het toevoegen van de metadata auditfile over alle regels van de dataset niet het meest efficient vinden.
    #In mijn geval wil ik de data van de auditfile gebruiken om een tableau workbook te maken met dashboards.
    
    temp_6 = temp_5.merge(af_info, on='file', how = 'left')

   # plak de auditfile info aan de transacties

    # voeg nog wat velden toe
    temp_6['effMonth'] = temp_6.effDate.map(lambda x: x.month) # omdat effdate een datumveld is kunnen we met een korte functie het maandnummer ontsluiten
    temp_6['grootboek'] = temp_6.accID + ' - ' + temp_6.accDesc
    temp_6['dagboek'] = temp_6.jrnID + ' - ' + temp_6.jrn_desc

    auditfile = temp_6

In [33]:
auditfile.groupby(['companyName', 'softwareDesc'])['amount'].sum().round(-2) # check of de trainsacties in balans zijn


companyName    softwareDesc
Toro Rosso BV  Exact Online    0.0
Name: amount, dtype: float64

***
# 5. Auditfile exporteren

In [35]:
writer = pd.ExcelWriter('auditfile.xlsx') # de auditfile kunnen we exporteren naar excel om vervolgens te gebruiken voor je doeleinden.
auditfile.to_excel(writer, 'export_af')
writer.save()

In [ ]:
# end of script